In [1]:
import pandas as pd
import numpy as np

In [2]:
gen = ["packaging_assembly", "pvp", "nucleotide_metabolism", "RNA-associated", "DNA-associated", "lysis", 
                  "cell_wall_depolymerase", "super_infection", "toxin", "anti-restriction", "crispr", 
                  "sir2", "transferase", "reductase", "adsorption-related", "phosphorylation", "ejection"]

In [5]:
all_preds = pd.read_csv("EFAM/empathi_predictions_EFAM.csv", index_col=0)

In [ ]:
all_preds_filt = all_preds.loc[(all_preds.loc[:, gen] > 0.95).sum(axis=1) >= 1]

# Compare annotated fraction

In [59]:
df2 = pd.read_csv("EFAM/Final_Super_Condensed_Annotations-updated_efam.tsv", sep="\t")
prot2clust = df2.loc[:, ["Cluster", "Annotation Status", "Proteins"]]
prot2clust.Proteins = prot2clust.Proteins.str.replace('[',"").str.replace("]","").str.replace("'","").str.replace(" ", "").str.split(',')
prot2clust = prot2clust.explode("Proteins")
prot2clust = prot2clust.set_index("Proteins")
prot2clust.index.name = None
#prot2clust.index = prot2clust.index.str.replace("'","").str.replace(" ", "")

In [7]:
compare = pd.merge(prot2clust, all_preds, left_index=True, right_index=True, how="outer")

print(compare.Cluster.isna().sum(), "sequences not in EFAM final annotations file.")
compare = compare.loc[~(compare.Cluster.isna())]

445417 sequences not in EFAM final annotations file.


In [5]:
def eval_clust(df):
    prevAnnotated = df["Annotation Status"].iloc[0] == "Annotated"
    empAnnotated = (df.Annotation != "unknown").any()
    
    #if (df.Annotation == "unknown").any():
    #    display(df.Annotation)
    #    print((df.Annotation != "unknown").any())

    if prevAnnotated & empAnnotated:
        return "Cluster annotated by both"
    if (not prevAnnotated) & empAnnotated:
        return "Cluster only annotated by Empathi"
    if prevAnnotated & (not empAnnotated):
        return "Cluster only annotated by EFAM"
    if (not prevAnnotated) & (not empAnnotated):
        return "Cluster not annotated"

In [39]:
print("Eval at 50% conf.")
evaled = compare.groupby("Cluster").apply(eval_clust)
evaled.value_counts()

Eval at 50% conf.


Cluster only annotated by Empathi    133948
Cluster annotated by both             72422
Cluster not annotated                 25932
Cluster only annotated by EFAM         8009
Name: count, dtype: int64

In [8]:
print("Eval at 95% conf.")
compare2 = compare.copy()
compare2["Annotation"] = compare2.Annotation.where((compare2.loc[:, gen] > 0.95).sum(axis=1) >= 1, "unknown")
evaled = compare2.groupby("Cluster").apply(eval_clust)
evaled.value_counts()

Eval at 95% conf.


/tmp/ipykernel_163700/1975702434.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  evaled = struct2.groupby("Cluster").apply(eval_clust)


Cluster not annotated                87151
Cluster only annotated by Empathi    72729
Cluster annotated by both            66056
Cluster only annotated by EFAM       14375
Name: count, dtype: int64

In [32]:
print(72729+66056, "clusters annotated by Empathi at 95% confidence.")

138785 clusters annotated by Empathi at 95% confidence.


In [14]:
list_prev_unknown = list(evaled.loc[evaled == "Cluster only annotated by Empathi"].index)

In [9]:
list_prev_known = list(evaled.loc[(evaled == "Cluster only annotated by EFAM") | (evaled == "Cluster annotated by both")].index)

# Entropy of clusters

In [35]:
def cols_greater_than_index(row):
    return "|".join(list(tmp.columns[row > 0.95]))

# Apply the function to each row
#tmp = struct2.drop(columns=["Cluster", "Annotation Status", "Annotation"])
#annos = tmp.apply(cols_greater_than_index, axis=1)

#struct2 = pd.merge(struct.loc[:, ["Cluster", "Annotation Status"]], tmp, left_index=True, right_index=True)
#annos.name = "Annotation"
#struct2 = pd.merge(annos, struct2, left_index=True, right_index=True)
#struct2.Annotation = struct2.Annotation.replace("", "unknown")

def redefine_annos(df):
    tmp = df.drop(columns=["Cluster", "Annotation Status", "Annotation"])
    annos = tmp.apply(cols_greater_than_index, axis=1)

    df = df.drop(columns=["Annotation"])
    annos.name = "Annotation"
    df = pd.merge(annos, df, left_index=True, right_index=True)
    df.Annotation = df.Annotation.replace("", "unknown")

    return df

In [12]:
def count_diff(df):
    if len(df.Annotation.unique()) == 1: 
        if len(df) == 1:
            return "singleton cluster" #singleton cluster
        else:
            return "all same anno" #all prots in cluster have same anno
    elif (len(df.Annotation.unique()) == 2) & ("unknown" in df.Annotation.unique()): 
        return "all same anno + unknown" #all prots have same anno with at least one "unknown"
    else:
        list_unique = pd.Series(df.Annotation.unique()).str.split("|")
        
        all_flags = []
        for i in list_unique:
            if "unknown" in i:
                pass
            else:
                for j in i:
                    flag=True
                    for k in list_unique:
                        if k == ["unknown"]:
                            pass
                        elif j not in k:
                            flag=False
                    all_flags.append(flag)
                    
                if np.array(all_flags).any():
                    return "at least one common anno" # all clusters recieved at least 1 common anno with differences in secondary predictions
                else:
                    return "at least 2 proteins with different anno" # at least 2 proteins in cluster recieved completely different annotations

#### On all clusters

In [35]:
counts = compare2.groupby("Cluster").apply(count_diff)
display(counts.value_counts())

/tmp/ipykernel_226437/2208859661.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  counts = struct2.groupby("Cluster").apply(count_diff)


all same anno                              170326
at least one common anno                    32720
all same anno + unknown                     30013
at least 2 proteins with different anno      7252
Name: count, dtype: int64

#### On previously unknown clusters

In [41]:
liste = list(evaled.loc[evaled == "Cluster annotated by both"].index)
preds = pd.merge(prot2clust, all_preds_filt, left_index=True, right_index=True)
preds = redefine_annos(preds)

tmp = preds.loc[~preds.Cluster.isin(list_prev_known)] #previously unknown clusters
counts = tmp.groupby("Cluster").apply(count_diff)
display(counts.value_counts())

/tmp/ipykernel_163700/4102927404.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  counts = tmp.groupby("Cluster").apply(count_diff)


all same anno                              44669
at least one common anno                   17138
singleton cluster                           9995
at least 2 proteins with different anno      927
Name: count, dtype: int64

#### On previously unknown clusters

In [42]:
liste = list(evaled.loc[evaled == "Cluster annotated by both"].index)
preds = pd.merge(prot2clust, all_preds_filt, left_index=True, right_index=True)
preds = redefine_annos(preds)

tmp = preds.loc[preds.Cluster.isin(list_prev_known)] #previously unknown clusters
counts = tmp.groupby("Cluster").apply(count_diff)
display(counts.value_counts())

/tmp/ipykernel_163700/2303153914.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  counts = tmp.groupby("Cluster").apply(count_diff)


all same anno                              51856
at least one common anno                   12201
singleton cluster                           1600
at least 2 proteins with different anno      399
Name: count, dtype: int64

# Virion proteins

In [60]:
df4 = pd.read_csv("EFAM/Peptide_to_protein_mapping_efam.tsv", sep="\t")

In [9]:
virion = prot2clust.loc[prot2clust.index.isin(list(df4["Protein Name"])), :]

In [69]:
all_preds_filt = all_preds.loc[(all_preds.loc[:, gen] > 0.95).sum(axis=1) >= 1]
all_preds_filt = all_preds_filt.loc[all_preds_filt.index.isin(virion.index)]
print(len(all_preds_filt), "total virion proteins in mapping dataset.")
print(len(all_preds_filt.loc[all_preds_filt.pvp > 0.95]), "predicted as PVP.")
print(len(all_preds_filt.loc[(all_preds_filt.pvp < 0.95) & (all_preds_filt.pvp > 0.5)]), "proteins predicted as pvp with confidence below 0.95.")
print(len(all_preds_filt.loc[all_preds_filt.pvp < 0.5]), "proteins predicted as non-pvp") 
all_preds_filt.loc[all_preds_filt.pvp < 0.5].Annotation.value_counts()[:40]

29355 total virion proteins in mapping dataset.
29036 predicted as PVP.
5 proteins predicted as pvp with confidence below 0.95.
314 proteins predicted as non-pvp


Annotation
DNA-associated|nuclease                                                                         47
nucleotide_metabolism|reductase                                                                 42
DNA-associated|ejection                                                                         37
DNA-associated|DNA_polymerase                                                                   28
transferase                                                                                     19
packaging_assembly                                                                              16
DNA-associated                                                                                  14
DNA-associated|helicase                                                                         14
DNA-associated|terminase|packaging_assembly                                                     13
DNA-associated|transcriptional_regulator                                                        12

In [79]:
tmp = pd.merge(all_preds_filt, virion, left_index=True, right_index=True, how="inner").reset_index().drop_duplicates(subset="index").Cluster.nunique()
print(tmp, "clusters in metap dataset also present in original fasta file.")

2529 clusters in metap dataset also present in original fasta file.


In [86]:
tmp = pd.merge(all_preds_filt, virion, left_index=True, right_index=True, how="inner").reset_index().drop_duplicates(subset="index")
print(tmp.loc[tmp.pvp > 0.95].Cluster.nunique(), "clusters predicted as PVP.")

2400 clusters predicted as PVP.


In [88]:
print(2529 - 2400, "clusters not predicted as PVP")

129 clusters not predicted as PVP
